# Data Preprocessing

## 1 - Basic Analysis

### 1.1 - Import Packages

In [32]:
import pandas as pd

# import tensorflow as tf
# from tensor flow import keras
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer

### 1.2 - Reading Data

In [2]:
# Read data from local pathway

project_path = "/Users/apple/Documents/Xinzhi/Data/"
data = pd.read_csv(project_path + "meta_training_data_jd_0401.csv")

data.head(3)

,Type,Code,fun1_code,fun2_code,jobname_ch,jobname_en,id,coname,poname,jd,source
0,FUN,BRO,BRO,BRO,商业研究,Business Research,279011,Aldevron,"Strategic Account Manager, Nucleic Acids Busin...",\n\nWe are currently seeking a Strategic Accou...,monster
1,FUN,BRO,BRO,BRO,商业研究,Business Research,287601,Synergy America,IT Business Analyst,\n\nWe are looking for an IT Business Analyst ...,monster
2,FUN,BRO,BRO,BRO,商业研究,Business Research,340361,A3 Smart Home,Sr. Business Analyst,"\n\nWe call our club's vision, mission, values...",monster


### 1.3 - Data Exploration

In [3]:
print("The number of tier 1 job title: ", len(data.fun1_code.unique()))
print("The number of tier 2 job title: ", len(data.Code.unique()))

The number of tier 1 job title:  97
The number of tier 2 job title:  709


In [4]:
# Class 'Code' that only have 1 instance
jd_count = data.groupby('Code').count()
jd_count[jd_count['jd']==1]

,Type,fun1_code,fun2_code,jobname_ch,jobname_en,id,coname,poname,jd,source
Code,,,,,,,,,,
AFB080,1,1,1,1,1,1,1,1,1,1
AFY660,1,1,1,1,1,1,1,1,1,1
AHR165,1,1,1,1,1,1,1,1,1,1
AMS900,1,1,1,1,1,1,1,1,1,1
APM080,1,1,1,1,1,1,1,1,1,1
APM900,1,1,1,1,1,1,1,1,1,1
AVO060,1,1,1,1,1,1,1,1,1,1
BNV,1,1,1,1,1,1,1,1,1,1
YHF020,1,1,1,1,1,1,1,1,1,1


In [5]:
jd_list = data.jd.tolist()

### 1.4 - Test  `text_to_word_sequence`

In [18]:
test1 = data.jd[20698]
# test1

In [19]:
test1_seq = text_to_word_sequence(test1)

In [20]:
tttk = Tokenizer()

In [21]:
tttk.fit_on_sequences(test1_seq)

In [23]:
tttk.document_count

1167

In [26]:
new_sep = []
for i in test_text:
    new_sep.append(text_to_word_sequence(i))

In [29]:
test1_tk = TextTokenizer(new_sep)

In [31]:
# test1_tk.tokenizer.word_index

### 1.5 - Test Keras Tokenizer

In [28]:
class TextTokenizer():
    def __init__(self, texts=None):
        self.texts = texts   
        self.tokenizer = Tokenizer()
        if texts is not None:
            self.tokenizer.fit_on_texts(texts)
            
#         tokenizer.word_counts
#         tokenizer.word_docs
#         tokenizer.word_index
#         tokenizer.document_count
        
    def train(self, train_text):
        self.tokenizer.fit_on_texts(train_text)

    def to_sequences(self, in_text=None):
        if in_text is None:
            return self.tokenizer.texts_to_sequences(self.texts)
        else:
            return self.tokenizer.texts_to_sequences(in_text)
    
    def max_length(self):
        sequences = self.to_sequences()
        lenth = []
        for i in sequences:
            lenth.append(len(i))
        mx_lenth = max(lenth)
        
        print(lenth.index(mx_lenth), mx_lenth)
    
    
    def max_length2(self):  
        sequences = self.to_sequences()
        lenth = 0
        for i in sequences:
            if len(i) > lenth:
                lenth = len(i)
                idx = sequences.index(i)
            else:
                continue

        print(idx, lenth)
    

In [57]:
test_text = [data.jd[20698], data.jd[2069]]
# test_text

In [ ]:
test_tk = TextTokenizer(test_text)

In [ ]:
# print(test_tk.tokenizer.word_index)

In [ ]:
# print(test_tk.to_sequences())

### Test JD

In [10]:
jd_tk = TextTokenizer(jd_list)

In [11]:
jd_tk.max_length()

# 45832 4219

45832 4219


In [12]:
jd_tk.max_length2()

# 45832 4219

45832 4219


## 2 - Cleaning

In [68]:
def segment_list(text_list):
    result = []
    for idx, text in enumerate(text_list):
        result[idx] = text_to_word_sequence(text)
    
    return result

In [69]:
def stem_list(word_list):
    result = []
    ps = PorterStemmer()
    for idx, word in enumerate(word_list):
        result[idx] = ps.stem(word)
    
    return result

In [70]:
def stem_sequence(seq_list):
    result = []
    for idx, seq in enumerate(seq_list):
        result[idx] = stem_list(seq)
    
    return result

In [63]:
# def stem_list(seq_list):
#     ps = PorterStemmer()
#     for seq in seq_list:
#         for idx, word in enumerate(seq):
#             seq[idx] = ps.stem(word)
    
#     return seq_list

In [60]:
def get_feature_count(text_list):
    sl = segment_list(text_list)
    tk = TextTokenizer(sl)
    
    return len(tk.tokenizer.word_counts)

In [71]:
def test_stem(text_list):
    print("Before stem: ", get_feature_count(text_list))
    stemmed = stem_sequence(text_list)
    print("After stem: ", get_feature_count(stemmed))

In [72]:
test_stem(test_text)

IndexError: list assignment index out of range

In [59]:
len(get_feature_count(test_text))

659

In [51]:
test_seg = segment_list(test_text)